In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
DATA_DIR = "data/oasis_longitudinal.csv"
df = pd.read_csv(DATA_DIR)

In [3]:
df = df.dropna(axis=0)
df.head(10)

,Subject ID,MRI ID,Group,Visit,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,OAS2_0001,OAS2_0001_MR2,Nondemented,2,457,M,R,88,14,2.0,30.0,0.0,2004,0.681,0.876
5,OAS2_0004,OAS2_0004_MR1,Nondemented,1,0,F,R,88,18,3.0,28.0,0.0,1215,0.710,1.444
6,OAS2_0004,OAS2_0004_MR2,Nondemented,2,538,F,R,90,18,3.0,27.0,0.0,1200,0.718,1.462
7,OAS2_0005,OAS2_0005_MR1,Nondemented,1,0,M,R,80,12,4.0,28.0,0.0,1689,0.712,1.039
8,OAS2_0005,OAS2_0005_MR2,Nondemented,2,1010,M,R,83,12,4.0,29.0,0.5,1701,0.711,1.032
9,OAS2_0005,OAS2_0005_MR3,Nondemented,3,1603,M,R,85,12,4.0,30.0,0.0,1699,0.705,1.033
13,OAS2_0008,OAS2_0008_MR1,Nondemented,1,0,F,R,93,14,2.0,30.0,0.0,1272,0.698,1.380
14,OAS2_0008,OAS2_0008_MR2,Nondemented,2,742,F,R,95,14,2.0,29.0,0.0,1257,0.703,1.396
15,OAS2_0009,OAS2_0009_MR1,Demented,1,0,M,R,68,12,2.0,27.0,0.5,1457,0.806,1.205


In [4]:
# Dataset Definition
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
class AlzheimersDataset(Dataset):
    
    def __init__(self, root_dir, transform=None):
        self.df = pd.read_csv(root_dir)
        cols_to_drop = ['Visit', 'Subject ID', 'MRI ID', 'Hand', 'M/F']
        self.df = self.df.dropna(axis=0)
        self.df = self.df.drop(cols_to_drop, axis=1)
        labels = [1 if group == 'Demented' else 0 for group in self.df.Group]
        self.df.Group = labels
        self.data = self.df.to_numpy()
    def __getitem__(self, i):
        return self.data[i]
    def __len__(self):
        return self.data.shape[0]

In [5]:
def generate_batch(batch):
    label = [entry[0] for entry in batch]
    inputs = [entry[1:] for entry in batch]
    return inputs, label

In [6]:
alzh_data = AlzheimersDataset(DATA_DIR)
train_len = int(len(alzh_data) * 0.9) # 10% Split
train_dataset, test_dataset = random_split(alzh_data, \
                                           [train_len, \
                                            len(alzh_data) - train_len
                                           ])
print("train_dataset length: {0} | test_dataset length: {1}".format(len(train_dataset), \
                                                                 len(test_dataset)))

train_dataset length: 318 | test_dataset length: 36


## Training SVM

In [7]:
# SVM
from sklearn.svm import SVC
def train_SVM(x, y, params):
    clf = SVC(**params)
    clf.fit(x, y)
    return clf

In [8]:
svm_params = {
    'C': 1,
    'kernel': 'poly',
    'gamma': 'auto',
    'degree': 2
}
train_x, train_y = generate_batch(train_dataset)
test_x, test_y = generate_batch(test_dataset)

In [9]:
clf = train_SVM(train_x, train_y, svm_params)

In [10]:
from sklearn.metrics import confusion_matrix
def get_error_rates(clf, x, y_true):
    y_pred = clf.predict(x)
    cm = confusion_matrix(y_true, y_pred)
#     print(cm)
    tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()
    fpr = fp/(fp+tn)
    fnr = fn/(tp+fn)
    return fpr, fnr, cm
    
fpr_svm, fnr_svm, cm_svm = get_error_rates(clf, train_x, train_y)
print("train_score: {0:.2f}%, test_score: {1:.2f}%".format(clf.score(train_x, train_y), \
                                                clf.score(test_x, test_y)))
print('fpr: {0:.2f}%, fnr: {0:.2f}%'.format( \
                                           fpr_svm, fnr_svm))

train_score: 0.95%, test_score: 0.97%
fpr: 1.00%, fnr: 1.00%


#### Prediction using Linear SVM

In [11]:
labels = {
    1:'Demented',
    0:'Nondemented'
}
idx = 60
prediction = clf.predict(np.array([alzh_data[idx, 1:]]))
print("Predicted: {}".format(labels[int(prediction)]))
print("Actual: {}".format(labels[alzh_data[idx, 0]]))

Predicted: Nondemented
Actual: Nondemented


## Training a Linear NN

In [12]:
# Models
# - SVM
# - Linear NN
import torch
import torch.nn as nn
import torch.nn.functional as F

class AlzheimersNN(nn.Module):
    
    def __init__(self, input_size, embed_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_size, embed_dim)
        self.linear = nn.Linear(embed_dim, 2)
        self.sigmoid = nn.Sigmoid()
        self.init_weights()
    
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.linear.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
    
    def forward(self, inp):
        embeds = self.embedding(inp)
        out = self.linear(embeds)
#         out = self.sigmoid(out)
        return out

In [13]:
def generate_batch_tensor(batch):
    label = torch.tensor([entry[0] for entry in batch])
    inputs = torch.tensor([entry[1:] for entry in batch]).long()
    return inputs, label

In [14]:
def train_func(sub_train_):
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True, \
                      collate_fn=generate_batch_tensor)
    for i, (inputs, label) in enumerate(data):
        optimizer.zero_grad()
        inputs, label = inputs.to(device), label.to(device)
        
        output = model(inputs)
        loss = criterion(output, label)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == label).sum().item()
    
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

In [69]:
import time
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# BATCH_SIZE = 4
# EMBED_DIM = 16
# N_EPOCHS = 5

# train_losses = []
# valid_losses = []

# model = AlzheimersNN(10, EMBED_DIM).to(device)
# criterion = torch.nn.CrossEntropyLoss().to(device)
# optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

# train_len = int(len(train_dataset) * 0.95)
# sub_train_, sub_valid = \
#     random_split(train_dataset, [train_len, len(train_dataset) - train_len])
# print("Training on {}".format(device))

# for epoch in range(N_EPOCHS):

#     start_time = time.time()
#     train_loss, train_acc = train_func(sub_train_)
#     valid_loss, valid_acc = test(sub_valid)
#     train_losses.append(train_loss)
#     val_losses.append(valid_loss)
#     secs = int(time.time() - start_time)
#     mins = secs / 60
#     secs = secs % 60

#     print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
#     print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
#     print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

# torch.save(model, "saved_models/linear_nn.pth")